# Extract HIP Cells and Create Expression Matrix for Deconvolution

This notebook extracts HIP (hippocampus) cells from GSE185862 10X scRNA-seq data and creates an expression matrix for deconvolution analysis.

**Note:** Data is already pre-processed, so QC steps are skipped.

**Workflow:**
1. Load HDF5 data and create AnnData object
2. Cell Annotation and Subsetting (remove unknown/unwanted cell types)
3. Normalization (after subsetting, similar to Seurat)
4. Find DEGs (Differentially Expressed Genes) per cell type
5. Create expression matrix using DEGs for deconvolution

**Reference:** Similar workflow to human scRNA-seq deconvolution pipeline


In [1]:
# Import required libraries
import h5py
import numpy as np
import pandas as pd
import anndata as ad
from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse import vstack as sparse_vstack
from scipy.io import mmwrite
import scipy.io
import os
from tqdm import tqdm
import scanpy as sc
import warnings
warnings.filterwarnings('ignore')

# Set scanpy settings
sc.settings.verbosity = 3  # verbosity level
sc.settings.set_figure_params(dpi=80, facecolor='white')

print("Libraries imported successfully!")
print(f"anndata version: {ad.__version__}")
print(f"scanpy version: {sc.__version__}")


Libraries imported successfully!
anndata version: 0.11.4
scanpy version: 1.11.5


## Step 1: Set File Paths and Load Metadata


In [2]:
# File paths
hdf5_path = "/data/project/1_Epilepsy_RNA/scRNA_Animal/Raw_Data/Animal_GSE185862/10X/expression_matrix.hdf5"
metadata_path = "/data/project/1_Epilepsy_RNA/scRNA_Animal/Raw_Data/Animal_GSE185862/10X/metadata_HIP_only.csv"
output_dir = "/home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv"

# Create output directories
os.makedirs(output_dir, exist_ok=True)
os.makedirs(os.path.join(output_dir, "01.Annotation"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "02.DEGs"), exist_ok=True)
os.makedirs(os.path.join(output_dir, "03.Matrix"), exist_ok=True)

print(f"Output directory: {output_dir}")
print(f"Directories created: ✓")
print("Note: Data is pre-processed, so QC steps are skipped.")


Output directory: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv
Directories created: ✓
Note: Data is pre-processed, so QC steps are skipped.


In [3]:
# Load HIP metadata
print("Loading HIP metadata...")
metadata = pd.read_csv(metadata_path)
print(f"  ✓ Loaded {len(metadata):,} HIP cells from metadata")
print(f"  Columns: {list(metadata.columns)}")

# Get HIP cell barcodes (sample_name column)
hip_cell_barcodes = set(metadata['sample_name'].values)
print(f"  ✓ Found {len(hip_cell_barcodes):,} unique HIP cell barcodes")

# Check key columns
if 'cluster_label' in metadata.columns:
    print(f"  ✓ Cell type annotations available: cluster_label, class_label")
    print(f"  Unique cell types: {metadata['cluster_label'].nunique()}")
    print(f"  Unique classes: {metadata['class_label'].nunique()}")
else:
    print("  ⚠ Warning: cluster_label not found in metadata")

metadata.head()


Loading HIP metadata...
  ✓ Loaded 82,535 HIP cells from metadata
  Columns: ['sample_name', 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_lab

,sample_name,donor_sex_id,donor_sex_label,donor_sex_color,region_id,region_label,region_color,platform_label,cluster_order,cluster_label,...,cell_type_alt_alias_id,cell_type_alias_id,cell_type_accession_id,cell_type_designation_color,cell_type_alt_alias_color,cell_type_alias_color,cell_type_accession_color,cortical_layer_label,cortical_layer_order,cortical_layer_color
0,CCAATCCAGCCACCTG-L8TX_180406_01_F07,2,M,#ADC4C3,28,HIP,#8DC63F,10X,10,10_Lamp5,...,10,10,10,#8C5962,#8C5962,#8C5962,#8C5962,NaN,1,#FF7373
1,CGAGCCAAGATGCGAC-L8TX_180221_01_E12,1,F,#565353,28,HIP,#8DC63F,10X,10,10_Lamp5,...,10,10,10,#8C5962,#8C5962,#8C5962,#8C5962,NaN,1,#FF7373
2,CTTCTCTGTAGAAGGA-L8TX_200625_02_G11,1,F,#565353,28,HIP,#8DC63F,10X,11,11_Lamp5,...,11,11,11,#9E686A,#9E686A,#9E686A,#9E686A,NaN,1,#FF7373
3,CAAGGCCAGTGAAGAG-L8TX_180406_01_G07,2,M,#ADC4C3,28,HIP,#8DC63F,10X,11,11_Lamp5,...,11,11,11,#9E686A,#9E686A,#9E686A,#9E686A,NaN,1,#FF7373
4,TCAGGATCAGGATTGG-L8TX_200625_02_G11,1,F,#565353,28,HIP,#8DC63F,10X,11,11_Lamp5,...,11,11,11,#9E686A,#9E686A,#9E686A,#9E686A,NaN,1,#FF7373


## Step 2: Load HDF5 Data and Extract HIP Cells


In [4]:
# Read HDF5 file and identify HIP cell indices
print("Reading HDF5 file and identifying HIP cells...")
print("  (This may take a few minutes for large files)...")

# Open HDF5 file
f = h5py.File(hdf5_path, 'r')
data_group = f['data']

# Read gene names
print("  - Reading gene names...")
gene_names = data_group['gene'][:]
if gene_names.dtype.kind == 'S':
    gene_names = np.array([g.decode('utf-8') for g in gene_names])
else:
    gene_names = np.array(gene_names)
n_genes = len(gene_names)
print(f"    ✓ Loaded {n_genes:,} genes")

# Read cell barcodes
print("  - Reading cell barcodes...")
cell_barcodes = data_group['samples'][:]
if cell_barcodes.dtype.kind == 'S':
    cell_barcodes = np.array([c.decode('utf-8') for c in cell_barcodes])
else:
    cell_barcodes = np.array(cell_barcodes)
n_cells_total = len(cell_barcodes)
print(f"    ✓ Loaded {n_cells_total:,} total cells")

# Find indices of HIP cells
print("  - Finding HIP cell indices...")
hip_indices = np.where(np.isin(cell_barcodes, list(hip_cell_barcodes)))[0]
n_hip_cells = len(hip_indices)
print(f"    ✓ Found {n_hip_cells:,} HIP cells ({n_hip_cells/n_cells_total*100:.2f}% of total)")

# Sort indices for efficient reading
hip_indices_sorted = np.sort(hip_indices)
print(f"  ✓ Indices sorted for efficient reading")


Reading HDF5 file and identifying HIP cells...
  (This may take a few minutes for large files)...
  - Reading gene names...
    ✓ Loaded 31,053 genes
  - Reading cell barcodes...
    ✓ Loaded 1,169,320 total cells
  - Finding HIP cell indices...
    ✓ Found 82,535 HIP cells (7.06% of total)
  ✓ Indices sorted for efficient reading


In [5]:
# Extract HIP cell expression matrix in chunks (memory efficient)
print("Extracting expression matrix for HIP cells...")
print("  (Reading counts matrix - this may take several minutes)...")

# Read counts matrix in chunks
counts_ds = data_group['counts']
chunk_size = 5000  # Process in chunks to manage memory
hip_counts_list = []

print(f"  - Reading {n_hip_cells:,} cells × {n_genes:,} genes in chunks...")
for i in tqdm(range(0, n_hip_cells, chunk_size), desc="  Progress"):
    end_idx = min(i + chunk_size, n_hip_cells)
    batch_indices = hip_indices_sorted[i:end_idx]
    
    # Read batch: transpose to get (cells × genes)
    # Counts in HDF5 are (genes × cells), so we transpose
    batch_counts = counts_ds[:, batch_indices].T.astype(np.float32)
    batch_counts_sparse = csr_matrix(batch_counts)
    hip_counts_list.append(batch_counts_sparse)

# Concatenate all batches using sparse vstack
print("  - Concatenating batches (sparse)...")
if len(hip_counts_list) > 1:
    hip_counts = sparse_vstack(hip_counts_list, format='csr')
else:
    hip_counts = hip_counts_list[0]

print(f"    ✓ Expression matrix shape: {hip_counts.shape} (cells × genes)")
print(f"    ✓ Non-zero elements: {hip_counts.nnz:,}")
print(f"    ✓ Sparsity: {(1 - hip_counts.nnz / hip_counts.size) * 100:.2f}%")


Extracting expression matrix for HIP cells...
  (Reading counts matrix - this may take several minutes)...
  - Reading 82,535 cells × 31,053 genes in chunks...


  Progress: 100%|██████████| 17/17 [01:09<00:00,  4.09s/it]


  - Concatenating batches (sparse)...
    ✓ Expression matrix shape: (82535, 31053) (cells × genes)
    ✓ Non-zero elements: 265,077,789
    ✓ Sparsity: 0.00%


In [6]:
# Match metadata with expression matrix order
print("Matching metadata with expression matrix...")
hip_barcodes_ordered = cell_barcodes[hip_indices_sorted]

# Match with metadata - ensure order is preserved
metadata_idx = metadata.set_index('sample_name')
hip_metadata = metadata_idx.loc[hip_barcodes_ordered].reset_index()

# Verify order
if len(hip_metadata) != n_hip_cells:
    print(f"  ⚠ Warning: Metadata mismatch ({len(hip_metadata)} vs {n_hip_cells} cells)")
else:
    if np.array_equal(hip_metadata['sample_name'].values, hip_barcodes_ordered):
        print("  ✓ Verified: Metadata order matches expression matrix")
    else:
        print("  ⚠ Warning: Barcode order mismatch! Reordering metadata...")
        hip_metadata = hip_metadata.set_index('sample_name').loc[hip_barcodes_ordered].reset_index()
        print("  ✓ Metadata reordered")

print(f"  ✓ Matched {len(hip_metadata):,} cells with metadata")

# Close HDF5 file
f.close()
print("  ✓ HDF5 file closed")


Matching metadata with expression matrix...
  ✓ Verified: Metadata order matches expression matrix
  ✓ Matched 82,535 cells with metadata
  ✓ HDF5 file closed


In [7]:
# Create AnnData object
print("Creating AnnData object...")
adata = ad.AnnData(
    X=hip_counts,
    obs=hip_metadata.set_index('sample_name'),
    var=pd.DataFrame(index=gene_names)
)

print(f"  ✓ AnnData object created")
print(f"    Shape: {adata.shape} (cells × genes)")
print(f"    Genes: {adata.n_vars:,}")
print(f"    Cells: {adata.n_obs:,}")
print(f"    Obs columns: {list(adata.obs.columns)}")

adata


Creating AnnData object...
  ✓ AnnData object created
    Shape: (82535, 31053) (cells × genes)
    Genes: 31,053
    Cells: 82,535
    Obs columns: ['donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_t

AnnData object with n_obs × n_vars = 82535 × 31053
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'ce

In [8]:
# Display AnnData object summary
print("AnnData object created:")
print(f"  Shape: {adata.shape} (cells × genes)")
print(f"  Cells: {adata.n_obs:,}")
print(f"  Genes: {adata.n_vars:,}")
print(f"  Data type: {type(adata.X)}")
print(f"  Note: Data is pre-processed (raw counts in adata.X)")

# Check cell type annotations
if 'cluster_label' in adata.obs.columns:
    print("\n  Cell type annotation available:")
    print(f"    Unique cell types (cluster_label): {adata.obs['cluster_label'].nunique()}")
    print(f"    Cell type distribution:")
    print(adata.obs['cluster_label'].value_counts().head(20))
    
if 'class_label' in adata.obs.columns:
    print(f"\n    Unique classes (class_label): {adata.obs['class_label'].nunique()}")
    print(f"    Class distribution:")
    print(adata.obs['class_label'].value_counts())

adata


AnnData object created:
  Shape: (82535, 31053) (cells × genes)
  Cells: 82,535
  Genes: 31,053
  Data type: <class 'scipy.sparse._csr.csr_matrix'>
  Note: Data is pre-processed (raw counts in adata.X)

  Cell type annotation available:
    Unique cell types (cluster_label): 187
    Cell type distribution:
cluster_label
363_DG          46911
364_DG           6237
348_CA1-do       6171
362_DG           5032
7_Lamp5 Lhx6     1788
338_CA1          1393
346_CA1-do       1246
347_CA1-do       1142
339_CA1           854
337_CA1           711
361_DG            578
376_Astro         476
29_Ntng1 HPF      463
56_Vip HPF        406
354_CA3-ve        353
344_CA1           344
12_Lamp5          305
340_CA1           300
11_Lamp5          291
352_CA3-ve        285
Name: count, dtype: int64

    Unique classes (class_label): 3
    Class distribution:
class_label
Glutamatergic    73990
GABAergic         7106
Non-Neuronal      1439
Name: count, dtype: int64


AnnData object with n_obs × n_vars = 82535 × 31053
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'ce

In [9]:
# Create simplified cell type annotations (similar to CellType1 in human workflow)
# Based on the HIP cell type summary - 6 cell types excluding ExN_DG, ExN_CA, ExN_SUB

if 'cluster_label' in adata.obs.columns and 'class_label' in adata.obs.columns:
    print("Creating simplified cell type annotations (CellType1) - 6 cell types...")
    print("  Combining: Endo + SMC-Peri → Endo_M")
    
    # Define cell type mappings based on cluster_label and class_label
    adata.obs['CellType1'] = 'Unknown'
    
    # Excitatory neurons (Glutamatergic) 
    exn_cells = ['Glutamatergic']

    # Inhibitory neurons (GABAergic)
    inn_cells = ['GABAergic']
    
    # Non-neuronal cells
    oligo_cells = ['375_Oligo', '368_Oligo', '374_Oligo', '372_Oligo', '370_Oligo', '369_Oligo', '373_Oligo', '371_Oligo']
    OPC_cells = ['366_Oligo', '365_Oligo', '367_Oligo']
    astro_cells = ['378_Astro', '377_Astro', '376_Astro']
    micro_cells = ['387_Micro-PVM', '386_Micro-PVM', '388_Micro-PVM']
    endo_cells = ['383_VLMC', '379_Endo', '384_VLMC', '380_SMC-Peri', '382_SMC-Peri']
    
    adata.obs.loc[adata.obs['class_label'].isin(exn_cells), 'CellType1'] = 'ExN_M'
    adata.obs.loc[adata.obs['class_label'].isin(inn_cells), 'CellType1'] = 'InN_M'
    adata.obs.loc[adata.obs['cluster_label'].isin(oligo_cells), 'CellType1'] = 'Oligo_M'
    adata.obs.loc[adata.obs['cluster_label'].isin(OPC_cells), 'CellType1'] = 'OPC_M'
    adata.obs.loc[adata.obs['cluster_label'].isin(astro_cells), 'CellType1'] = 'Astro_M'
    adata.obs.loc[adata.obs['cluster_label'].isin(micro_cells), 'CellType1'] = 'Micro_M'
    adata.obs.loc[adata.obs['cluster_label'].isin(endo_cells), 'CellType1'] = 'Endo_M'
    
    print("  ✓ CellType1 annotations created")
    print("\n  CellType1 distribution (6 cell types):")
    print(adata.obs['CellType1'].value_counts())
    
    # Check for excluded and Unknown cells
    unknown_count = (adata.obs['CellType1'] == 'Unknown').sum()
    
    print(f"\n  Cells excluded (will be removed):")
    print(f"    Unknown/Other: {unknown_count:,} cells")
else:
    print("  ⚠ Warning: cluster_label or class_label not found. Cannot create CellType1 annotations.")


Creating simplified cell type annotations (CellType1) - 6 cell types...
  Combining: Endo + SMC-Peri → Endo_M
  ✓ CellType1 annotations created

  CellType1 distribution (6 cell types):
CellType1
ExN_M      73990
InN_M       7106
Astro_M      534
Oligo_M      347
Micro_M      225
OPC_M        213
Endo_M       120
Name: count, dtype: int64

  Cells excluded (will be removed):
    Unknown/Other: 0 cells


In [10]:
# Subset cells: Remove Unknown and excluded ExN subtypes (similar to human workflow)
# In human workflow: exclude "Imm" and "CR" cells
print("Subsetting cells of interest...")
print(f"  Before subsetting: {adata.n_obs:,} cells")

# Remove Unknown cells and excluded ExN subtypes (DG, CA, SUB)
# These are already marked as 'Unknown' in CellType1, so we just filter them out
adata_subset = adata[adata.obs['CellType1'] != 'Unknown', :].copy()

print(f"  After subsetting: {adata_subset.n_obs:,} cells")
print(f"  Removed: {adata.n_obs - adata_subset.n_obs:,} cells")

print("\n  Final CellType1 distribution (6 cell types):")
print(adata_subset.obs['CellType1'].value_counts())

# Update adata to the subset
adata = adata_subset.copy()

print(f"\n  ✓ Subsetting complete: {adata.n_obs:,} cells × {adata.n_vars:,} genes")
adata

Subsetting cells of interest...
  Before subsetting: 82,535 cells
  After subsetting: 82,535 cells
  Removed: 0 cells

  Final CellType1 distribution (6 cell types):
CellType1
ExN_M      73990
InN_M       7106
Astro_M      534
Oligo_M      347
Micro_M      225
OPC_M        213
Endo_M       120
Name: count, dtype: int64

  ✓ Subsetting complete: 82,535 cells × 31,053 genes


AnnData object with n_obs × n_vars = 82535 × 31053
    obs: 'donor_sex_id', 'donor_sex_label', 'donor_sex_color', 'region_id', 'region_label', 'region_color', 'platform_label', 'cluster_order', 'cluster_label', 'cluster_color', 'subclass_order', 'subclass_label', 'subclass_color', 'neighborhood_id', 'neighborhood_label', 'neighborhood_color', 'class_order', 'class_label', 'class_color', 'exp_component_name', 'external_donor_name_label', 'full_genotype_label', 'facs_population_plan_label', 'injection_roi_label', 'injection_materials_label', 'injection_method_label', 'injection_type_label', 'full_genotype_id', 'full_genotype_color', 'external_donor_name_id', 'external_donor_name_color', 'facs_population_plan_id', 'facs_population_plan_color', 'injection_materials_id', 'injection_materials_color', 'injection_method_id', 'injection_method_color', 'injection_roi_id', 'injection_roi_color', 'injection_type_id', 'injection_type_color', 'cell_type_accession_label', 'cell_type_alias_label', 'ce

## Step 4: Normalization

Normalize data after cell type subsetting (similar to Seurat's NormalizeData)

In [11]:
# Normalize data after subsetting (similar to Seurat's NormalizeData)
print("Normalizing data after cell type subsetting...")
print(f"  Before normalization: {adata.shape} (cells × genes)")

# Normalize to 10,000 UMI per cell (equivalent to Seurat NormalizeData with scale.factor = 10000)
sc.pp.normalize_total(adata, target_sum=1e4)  # Normalize to 10,000 UMI per cell
sc.pp.log1p(adata)  # Log transform (log(x + 1))

print("  ✓ Data normalized (equivalent to Seurat NormalizeData)")
print(f"    Normalized counts stored in adata.X")
print(f"    After normalization: {adata.shape} (cells × genes)")
print(f"    Normalization: Total counts per cell = 10,000, then log(x + 1) transform")

# Store normalized counts in layers for reference (optional)
adata.layers['counts'] = adata.X.copy()  # This will be normalized, but we'll use it for reference

print("  ✓ Normalization complete")

Normalizing data after cell type subsetting...
  Before normalization: (82535, 31053) (cells × genes)
normalizing counts per cell
    finished (0:00:00)
  ✓ Data normalized (equivalent to Seurat NormalizeData)
    Normalized counts stored in adata.X
    After normalization: (82535, 31053) (cells × genes)
    Normalization: Total counts per cell = 10,000, then log(x + 1) transform
  ✓ Normalization complete


## Step 5: Find Differentially Expressed Genes (DEGs)

Find cell-type-specific markers using scanpy's rank_genes_groups (equivalent to Seurat's FindAllMarkers)
- Use Wilcoxon rank-sum test (similar to Seurat)
- Select top 200 DEGs per cell type
- Create union of all DEGs

In [12]:
# Find DEGs per cell type (equivalent to FindAllMarkers in Seurat)
print("Finding differentially expressed genes per cell type...")
print("  Using Wilcoxon rank-sum test (similar to Seurat FindAllMarkers)")
print("  Using normalized data from adata.X")

# Set CellType1 as the grouping variable
adata.obs['CellType1'] = adata.obs['CellType1'].astype('category')

# Find markers for each cell type
# method='wilcoxon' is equivalent to Seurat's Wilcoxon Rank Sum test
# use_raw=False to use log-normalized data in adata.X
sc.tl.rank_genes_groups(
    adata, 
    groupby='CellType1',
    method='wilcoxon',
    use_raw=False,  # Use normalized data in adata.X
    n_genes=adata.n_vars  # Test all genes
)

print("  ✓ DEG analysis complete")
print(f"  Tested {adata.n_vars:,} genes across {adata.obs['CellType1'].nunique()} cell types")

Finding differentially expressed genes per cell type...
  Using Wilcoxon rank-sum test (similar to Seurat FindAllMarkers)
  Using normalized data from adata.X
ranking genes
    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:03:43)
  ✓ DEG analysis complete
  Tested 31,053 genes across 7 cell types


In [13]:
# Extract top markers per cell type (top 200, similar to human workflow)
top_markers_per_celltype = 200
print(f"Extracting top {top_markers_per_celltype} markers per cell type...")

# Get results as DataFrame
markers_df = sc.get.rank_genes_groups_df(adata, group=None)

# Filter for positive markers only (only.pos = TRUE in Seurat)
# In scanpy, scores can be negative, we'll filter by p-value and keep top by score
markers_df = markers_df[markers_df['pvals_adj'] < 0.05]  # Significant markers
markers_df = markers_df.sort_values(['group', 'scores'], ascending=[True, False])

# Select top N markers per cell type
markers_top_list = []
for cell_type in adata.obs['CellType1'].cat.categories:
    cell_type_markers = markers_df[markers_df['group'] == cell_type].head(top_markers_per_celltype)
    markers_top_list.append(cell_type_markers)

markers_top = pd.concat(markers_top_list, ignore_index=True)

print(f"  ✓ Selected top {top_markers_per_celltype} markers per cell type")
print(f"  Total markers: {len(markers_top):,}")

# Union of all marker genes (unique genes across all cell types)
union_markers = markers_top['names'].unique()
print(f"  ✓ Union of markers: {len(union_markers):,} unique genes")

# Save marker information
markers_output_dir = os.path.join(output_dir, "02.DEGs")
markers_top.to_csv(
    os.path.join(markers_output_dir, "GSE185862_HIP_markers_CellType1_top200.txt"),
    sep="\t", index=False
)
pd.DataFrame({'Gene': union_markers}).to_csv(
    os.path.join(markers_output_dir, "GSE185862_HIP_union_markers_CellType1.txt"),
    sep="\t", index=False, header=False
)

print(f"  ✓ Markers saved to {markers_output_dir}")

markers_top.head(20)

Extracting top 200 markers per cell type...
  ✓ Selected top 200 markers per cell type
  Total markers: 1,400
  ✓ Union of markers: 1,175 unique genes
  ✓ Markers saved to /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv/02.DEGs


,group,names,scores,logfoldchanges,pvals,pvals_adj
0,Astro_M,Mt1,39.521954,5.285946,0.000000e+00,0.000000e+00
1,Astro_M,Atp1a2,39.486462,10.280325,0.000000e+00,0.000000e+00
2,Astro_M,Cst3,39.438988,7.221253,0.000000e+00,0.000000e+00
3,Astro_M,Plpp3,39.423355,9.816142,0.000000e+00,0.000000e+00
4,Astro_M,Slc1a3,39.355793,7.813001,0.000000e+00,0.000000e+00
5,Astro_M,Apoe,39.342403,9.692268,0.000000e+00,0.000000e+00
6,Astro_M,Slc1a2,39.296940,6.616623,0.000000e+00,0.000000e+00
7,Astro_M,Mt2,39.041374,8.127082,0.000000e+00,0.000000e+00
8,Astro_M,Gpc5,37.972965,7.054075,0.000000e+00,0.000000e+00
9,Astro_M,Gja1,37.940693,12.021809,0.000000e+00,0.000000e+00


In [14]:
# Check marker distribution per cell type
print("Marker distribution per cell type:")
print(markers_top.groupby('group')['names'].count())

# Visualize top markers
print("Top 10 markers per cell type:")
for cell_type in sorted(adata.obs['CellType1'].cat.categories):
    top_10 = markers_top[markers_top['group'] == cell_type].head(10)
    print(f"  {cell_type}:")
    print(f"    {', '.join(top_10['names'].head(10).tolist())}")

Marker distribution per cell type:
group
Astro_M    200
Endo_M     200
ExN_M      200
InN_M      200
Micro_M    200
OPC_M      200
Oligo_M    200
Name: names, dtype: int64
Top 10 markers per cell type:
  Astro_M:
    Mt1, Atp1a2, Cst3, Plpp3, Slc1a3, Apoe, Slc1a2, Mt2, Gpc5, Gja1
  Endo_M:
    Utrn, Sparc, Epas1, Ebf1, Id3, Cobll1, Ftl1, Tpt1, Rbpms, Rbms1
  ExN_M:
    Ppp3ca, Nrgn, Celf2, Slit3, Chn1, Ptk2b, Camk2a, Cacna1e, Ryr2, Atp2b1
  InN_M:
    Grip1, Erbb4, Gad2, Slc6a1, Nap1l5, Gad1, Rbms3, Grik1, Dner, Dpp10
  Micro_M:
    Cst3, C1qc, C1qa, Tyrobp, Selenop, Ftl1, C1qb, Rps29, Csf1r, Serinc3
  OPC_M:
    Lhfpl3, Lrrc4c, Ptprz1, Pcdh15, Tnr, Gpc5, Luzp2, Dscam, Grid2, Brinp3
  Oligo_M:
    Plp1, Mbp, Tubb4a, Qk, Pde4b, Cryab, Gatm, Cnp, Cldn11, Scd2


In [15]:
# Filter expression matrix to include only DEG markers
print("Creating expression matrix with DEG markers...")

# Get expression matrix (using normalized data)
expression_matrix = adata.X  # This is log-normalized data
features = adata.var_names.values

# Check which markers are actually present in the expression matrix
markers_in_matrix = union_markers[ np.isin(union_markers, features) ]
missing_markers = set(union_markers) - set(markers_in_matrix)

if len(missing_markers) > 0:
    print(f"  ⚠ Warning: {len(missing_markers)} markers not found in expression matrix")
    print(f"    Missing markers: {list(missing_markers)[:10]}...")

print(f"  Markers in matrix: {len(markers_in_matrix):,} / {len(union_markers):,}")

# Filter expression matrix to DEG markers only
marker_indices = np.where(np.isin(features, markers_in_matrix))[0]
expression_matrix_filtered = expression_matrix[:, marker_indices]
features_filtered = features[marker_indices]

print(f"  ✓ Filtered expression matrix: {expression_matrix_filtered.shape}")
print(f"    Cells: {expression_matrix_filtered.shape[0]:,}")
print(f"    Genes (DEGs): {expression_matrix_filtered.shape[1]:,}")

Creating expression matrix with DEG markers...
  Markers in matrix: 1,175 / 1,175
  ✓ Filtered expression matrix: (82535, 1175)
    Cells: 82,535
    Genes (DEGs): 1,175


## Step 6: Create Expression Matrix for Deconvolution

Filter expression matrix to include only DEG markers and prepare for deconvolution tools

In [17]:
# Convert to DataFrame format similar to human workflow
# In human workflow, they create a matrix with Gene column and cell annotations as column headers
print("Preparing expression matrix for deconvolution output...")

# Verify cell count is exactly 82535
print(f"  Verifying cell count: {expression_matrix_filtered.shape[0]:,} cells expected")
print(f"  AnnData cells: {adata.n_obs:,}")
assert expression_matrix_filtered.shape[0] == adata.n_obs == 82535, f"Cell count mismatch: {expression_matrix_filtered.shape[0]} vs {adata.n_obs}"

# Convert sparse matrix to dense for output (if not too large)
# For large matrices, we might want to keep it sparse, but deconvolution tools often need dense
if expression_matrix_filtered.shape[0] * expression_matrix_filtered.shape[1] < 1e9:  # < 1 billion elements
    # Convert to dense array
    if hasattr(expression_matrix_filtered, 'toarray'):
        dense_array = expression_matrix_filtered.toarray()
    else:
        dense_array = expression_matrix_filtered
    
    # Verify shape before transposing: should be (82535, 1175) = (cells, genes)
    print(f"  Dense array shape: {dense_array.shape} (cells × genes)")
    assert dense_array.shape == (82535, 1175), f"Unexpected dense array shape: {dense_array.shape}"
    
    # Create DataFrame with genes as rows and cells as columns
    # Transpose: (82535, 1175) -> (1175, 82535) = (genes, cells)
    dense_array_T = dense_array.T
    print(f"  Transposed shape: {dense_array_T.shape} (genes × cells)")
    
    # Verify we have exactly 82535 cell columns
    assert dense_array_T.shape[1] == 82535, f"Expected 82535 cell columns, got {dense_array_T.shape[1]}"
    assert len(adata.obs_names) == 82535, f"Expected 82535 cell names, got {len(adata.obs_names)}"
    
    expression_df = pd.DataFrame(
        dense_array_T,  # Shape: (1175, 82535) = (genes, cells)
        index=features_filtered,  # 1175 gene names
        columns=adata.obs_names   # 82535 cell barcodes
    )
    
    # Verify DataFrame shape before adding Gene column
    assert expression_df.shape == (1175, 82535), f"DataFrame shape should be (1175, 82535), got {expression_df.shape}"
    
    # Add Gene column as first column (similar to human workflow)
    expression_df.insert(0, 'Gene', expression_df.index.values)
    
    # Verify shape after adding Gene column: should be (1175, 82536) = (genes, 1 Gene column + 82535 cell columns)
    assert expression_df.shape == (1175, 82536), f"After adding Gene column, shape should be (1175, 82536), got {expression_df.shape}"
    assert expression_df.columns[0] == 'Gene', "First column should be 'Gene'"
    assert len(expression_df.columns) == 82536, f"Total columns should be 82536 (1 Gene + 82535 cells), got {len(expression_df.columns)}"
    
    expression_df = expression_df.reset_index(drop=True)
    
    print(f"  ✓ Expression matrix converted to DataFrame")
    print(f"    Shape: {expression_df.shape} (genes × columns)")
    print(f"    Genes: {expression_df.shape[0]:,}")
    print(f"    Total columns: {expression_df.shape[1]:,} (1 Gene + {expression_df.shape[1]-1:,} cells)")
    print(f"    Cell columns: {expression_df.shape[1]-1:,}")
else:
    print("  ⚠ Warning: Matrix too large for dense format. Keeping sparse format.")
    expression_df = None

Preparing expression matrix for deconvolution output...
  Verifying cell count: 82,535 cells expected
  AnnData cells: 82,535
  Dense array shape: (82535, 1175) (cells × genes)
  Transposed shape: (1175, 82535) (genes × cells)
  ✓ Expression matrix converted to DataFrame
    Shape: (1175, 82536) (genes × columns)
    Genes: 1,175
    Total columns: 82,536 (1 Gene + 82,535 cells)
    Cell columns: 82,535


In [18]:
# Save expression matrix for deconvolution (similar to human workflow format)
# Reference file format: Gene column (first column) + cell type labels as column headers
matrix_output_dir = os.path.join(output_dir, "03.Matrix")

if expression_df is not None:
    # Verify the DataFrame structure
    assert expression_df.shape[0] == 1175, f"Expected 1175 genes, got {expression_df.shape[0]}"
    assert expression_df.shape[1] == 82536, f"Expected 82536 columns (1 Gene + 82535 cells), got {expression_df.shape[1]}"
    assert expression_df.columns[0] == 'Gene', "First column must be 'Gene'"
    
    # Get cell type annotations (one per cell, matching the order of cells in adata.obs_names)
    # The DataFrame columns are: 'Gene' + cell barcodes (adata.obs_names)
    # We need cell type labels in the same order as the cell barcodes
    celltype_annotations = adata.obs['CellType1'].values
    assert len(celltype_annotations) == 82535, f"Expected 82535 cell type annotations, got {len(celltype_annotations)}"
    
    # Create header: 'Gene' + cell type labels (matching reference file format)
    # Reference file format: Gene\tExN\tExN\tExN\t... (cell type labels as headers)
    header = ['Gene'] + celltype_annotations.tolist()
    assert len(header) == 82536, f"Header should have 82536 elements (1 Gene + 82535 cells), got {len(header)}"
    
    # Output file path (matching reference file naming convention)
    output_path = os.path.join(matrix_output_dir, "GSE185862_HIP_scRNA_matrix_DEGs_CellType1.txt")
    
    print(f"  Saving expression matrix to: {output_path}")
    print(f"    Format: Tab-separated, matching reference file format")
    print(f"    Genes (rows): {expression_df.shape[0]:,}")
    print(f"    Total columns: {expression_df.shape[1]:,} (1 Gene + {len(celltype_annotations):,} cells)")
    print(f"    Cell columns: {len(celltype_annotations):,}")
    
    # Save with proper header format (matching reference file)
    # Write header row first
    with open(output_path, 'w') as f:
        f.write('\t'.join(header) + '\n')
    
    # Append data rows (without writing the DataFrame column names again)
    # The DataFrame already has 'Gene' as the first column with gene names, so we just write the data
    import csv
    expression_df.to_csv(output_path, mode='a', sep='\t', index=False, header=False, quoting=csv.QUOTE_NONE)
    
    # Verify the saved file
    print(f"  ✓ Expression matrix saved successfully")
    print(f"    File: {output_path}")
    print(f"    Format: Tab-separated text file matching reference format")
    print(f"    Structure: Gene column (first) + 82535 cell columns with cell type labels as headers")
    
    # Also save as CSV for easier inspection (optional)
    csv_path = os.path.join(matrix_output_dir, "GSE185862_HIP_scRNA_matrix_DEGs_CellType1.csv")
    expression_df.to_csv(csv_path, index=False)
    print(f"    Also saved as CSV: {csv_path}")
    
else:
    # Save sparse format (if DataFrame creation failed)
    print("  Saving sparse format...")
    scipy.io.mmwrite(
        os.path.join(matrix_output_dir, "GSE185862_HIP_scRNA_matrix_DEGs_CellType1_sparse.mtx"),
        expression_matrix_filtered.T  # Transpose: genes × cells
    )
    # Save gene names and cell annotations
    pd.DataFrame({'Gene': features_filtered}).to_csv(
        os.path.join(matrix_output_dir, "GSE185862_HIP_DEG_genes.txt"),
        index=False, header=False
    )
    adata.obs[['CellType1']].to_csv(
        os.path.join(matrix_output_dir, "GSE185862_HIP_cell_annotations.txt"),
        sep="\t"
    )
    print(f"  ✓ Sparse matrix saved (due to large size)")

  Saving expression matrix to: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv/03.Matrix/GSE185862_HIP_scRNA_matrix_DEGs_CellType1.txt
    Format: Tab-separated, matching reference file format
    Genes (rows): 1,175
    Total columns: 82,536 (1 Gene + 82,535 cells)
    Cell columns: 82,535
  ✓ Expression matrix saved successfully
    File: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv/03.Matrix/GSE185862_HIP_scRNA_matrix_DEGs_CellType1.txt
    Format: Tab-separated text file matching reference format
    Structure: Gene column (first) + 82535 cell columns with cell type labels as headers
    Also saved as CSV: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv/03.Matrix/GSE185862_HIP_scRNA_matrix_DEGs_CellType1.csv


## Step 7: Save Final Objects

Save the processed AnnData object and expression matrices

In [19]:
# Save final AnnData object
print("Saving final AnnData object...")

output_h5ad = os.path.join(output_dir, "GSE185862_HIP_processed.h5ad")
adata.write(output_h5ad)
print(f"  ✓ AnnData object saved: {output_h5ad}")

# Copy the matrix file to parent directory with new name
print("\nCopying matrix file to parent directory...")
source_file = os.path.join(output_dir, "03.Matrix", "GSE185862_HIP_scRNA_matrix_DEGs_CellType1.txt")
target_file = os.path.join(output_dir, "GSE185862_10X_scRNA_matrix_DEGs_CellType1.txt")

if os.path.exists(source_file):
    import shutil
    shutil.copy2(source_file, target_file)
    print(f"  ✓ Matrix file copied:")
    print(f"    From: {source_file}")
    print(f"    To: {target_file}")
    
    # Verify the copied file
    if os.path.exists(target_file):
        file_size = os.path.getsize(target_file) / (1024**2)  # Size in MB
        print(f"    ✓ Verification: File copied successfully ({file_size:.2f} MB)")
    else:
        print(f"    ⚠ Warning: Target file not found after copying")
else:
    print(f"  ⚠ Warning: Source file not found: {source_file}")

print("\n" + "="*80)
print("PROCESSING COMPLETE!")
print("="*80)
print(f"Summary:")
print(f"  Input cells: {n_hip_cells:,} HIP cells (pre-processed data)")
print(f"  After subsetting: {adata.n_obs:,} cells")
print(f"  After normalization: {adata.n_obs:,} cells")
print(f"  Final genes: {adata.n_vars:,}")
print(f"  Cell types: {adata.obs['CellType1'].nunique()}")
print(f"  DEG markers: {len(union_markers):,} unique genes")
print(f"  Final matrix: {expression_matrix_filtered.shape}")
print(f"Output files saved to: {output_dir}")
print(f"Main matrix file: {target_file}")
print("="*80)

Saving final AnnData object...
  ✓ AnnData object saved: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv/GSE185862_HIP_processed.h5ad

Copying matrix file to parent directory...
  ✓ Matrix file copied:
    From: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv/03.Matrix/GSE185862_HIP_scRNA_matrix_DEGs_CellType1.txt
    To: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv/GSE185862_10X_scRNA_matrix_DEGs_CellType1.txt
    ✓ Verification: File copied successfully (915.37 MB)

PROCESSING COMPLETE!
Summary:
  Input cells: 82,535 HIP cells (pre-processed data)
  After subsetting: 82,535 cells
  After normalization: 82,535 cells
  Final genes: 31,053
  Cell types: 7
  DEG markers: 1,175 unique genes
  Final matrix: (82535, 1175)
Output files saved to: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv
Main matrix file: /home/joonho345/1_Epilepsy_RNA/scRNA_Animal/02.GSE185862_10X_Deconv/GSE185862_10X_scRNA_matrix_DEGs_C